# Tooth Segmentation with 3D U-Net

In [14]:

import nibabel as nib
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from skimage import measure
import vtk
import h5py
import pydicom


### Step 1: Load DICOM and Preprocess

In [15]:
def load_dicom_images(dicom_folder_path):
    """
    Load DICOM images from a directory and return them as a 3D numpy array.
    """
    # Collect all file paths in the folder with .dcm extension
    dicom_files = [os.path.join(dicom_folder_path, f) for f in os.listdir(dicom_folder_path) if f.endswith('.dcm')]
    
    # Sort files to ensure consistent order
    dicom_files = sorted(dicom_files)
    
    # Load each DICOM file, extract pixel array, and stack into a 3D volume
    images = [pydicom.dcmread(file).pixel_array for file in dicom_files]
    volume = np.stack(images, axis=-1)  # Stack along a new axis to create a 3D volume
    volume = np.array(volume, dtype=np.float32)  # Ensure consistent data type
    return volume

### Step 2: Preprocessing - Normalizing

In [16]:
def normalize(volume, min_bound=-1000, max_bound=400):
    """
    Normalize the volume between 0 and 1.
    """
    volume = (volume - min_bound) / (max_bound - min_bound)
    volume = np.clip(volume, 0, 1)
    return volume


In [17]:
# Load and normalize DICOM data
dicom_volume = load_dicom_images(r"D:\DIcom gans\Data\raw")  # Using raw string to handle backslashes in path
normalized_volume = normalize(dicom_volume)


### Step 3: Define 3D U-Net Model

In [18]:
import tensorflow as tf
from tensorflow.keras import layers, models

def unet_model(input_shape=(128, 128, 64, 1)):
    inputs = layers.Input(input_shape)
    
    # Encoder
    c1 = layers.Conv3D(32, 3, activation="relu", padding="same")(inputs)
    c1 = layers.BatchNormalization()(c1)
    c1 = layers.Conv3D(32, 3, activation="relu", padding="same")(c1)
    c1 = layers.BatchNormalization()(c1)
    p1 = layers.MaxPooling3D((2, 2, 2))(c1)

    c2 = layers.Conv3D(64, 3, activation="relu", padding="same")(p1)
    c2 = layers.BatchNormalization()(c2)
    c2 = layers.Conv3D(64, 3, activation="relu", padding="same")(c2)
    c2 = layers.BatchNormalization()(c2)
    p2 = layers.MaxPooling3D((2, 2, 2))(c2)
    p2 = layers.Dropout(0.3)(p2)  # Dropout for regularization

    c3 = layers.Conv3D(128, 3, activation="relu", padding="same")(p2)
    c3 = layers.BatchNormalization()(c3)
    c3 = layers.Conv3D(128, 3, activation="relu", padding="same")(c3)
    c3 = layers.BatchNormalization()(c3)

    # Decoder
    u1 = layers.Conv3DTranspose(64, 3, strides=(2, 2, 2), padding="same")(c3)
    u1 = layers.concatenate([u1, c2])
    c4 = layers.Conv3D(64, 3, activation="relu", padding="same")(u1)
    c4 = layers.BatchNormalization()(c4)
    c4 = layers.Conv3D(64, 3, activation="relu", padding="same")(c4)
    c4 = layers.BatchNormalization()(c4)

    u2 = layers.Conv3DTranspose(32, 3, strides=(2, 2, 2), padding="same")(c4)
    u2 = layers.concatenate([u2, c1])
    c5 = layers.Conv3D(32, 3, activation="relu", padding="same")(u2)
    c5 = layers.BatchNormalization()(c5)
    c5 = layers.Conv3D(32, 3, activation="relu", padding="same")(c5)
    c5 = layers.BatchNormalization()(c5)

    # Output Layer
    outputs = layers.Conv3D(1, 1, activation="sigmoid")(c5)

    model = models.Model(inputs, outputs)
    return model

# Instantiate the model
model = unet_model()
model.summary()

    

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 64, 1)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d (Conv3D)     │ (None, 128, 128,  │        896 │ input_layer[0][0] │
│                     │ 64, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 128, 128,  │        128 │ conv3d[0][0]      │
│ (BatchNormalizatio… │ 64, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_1 (Conv3D)   │ (None, 128, 128,  │     27,680 │ batch_normalizat… │
│                     │ 64, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        128 │ conv3d_1[0][0]    │
│ (BatchNormalizatio… │ 64, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d       │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (MaxPooling3D)      │ 32, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_2 (Conv3D)   │ (None, 64, 64,    │     55,360 │ max_pooling3d[0]… │
│                     │ 32, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        256 │ conv3d_2[0][0]    │
│ (BatchNormalizatio… │ 32, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_3 (Conv3D)   │ (None, 64, 64,    │    110,656 │ batch_normalizat… │
│                     │ 32, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        256 │ conv3d_3[0][0]    │
│ (BatchNormalizatio… │ 32, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_1     │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (MaxPooling3D)      │ 16, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 32, 32,    │          0 │ max_pooling3d_1[… │
│                     │ 16, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_4 (Conv3D)   │ (None, 32, 32,    │    221,312 │ dropout[0][0]     │
│                     │ 16, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        512 │ conv3d_4[0][0]    │
│ (BatchNormalizatio… │ 16, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_5 (Conv3D)   │ (None, 32, 32,    │    442,496 │ batch_normalizat… │
│                     │ 16, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        512 │ conv3d_5[0][0]    │
│ (BatchNormalizatio… │ 16, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_transpose    │ (None, 64, 64,    │    221,248 │ batch_normalizat

 Total params: 1,552,481 (5.92 MB)

 Trainable params: 1,551,201 (5.92 MB)

 Non-trainable params: 1,280 (5.00 KB)

In [19]:
import tensorflow as tf
import time
from sklearn.model_selection import train_test_split

# Assuming `input_volumes` and `target_volumes` are numpy arrays containing your data
# Split data into training and validation sets
train_X, val_X, train_y, val_y = train_test_split(input_volumes, target_volumes, test_size=0.2, random_state=42)

NameError: name 'input_volumes' is not defined